# Семинар №4

#### Семинар подготовлен с целью ознакомить слушателей с процедурой подготовки данных для алгоритмов машнинного обучения на примере задачи оценки цены недвижимости. Описание данных можно найти в data_description.txt

### 1. Считывание данных (1 балл)

Считайте данные из файлов train.csv и test.csv в массивы:
trainX (содержит признаки обучающего множества)
trainY (содержит правильные ответа для обучающего множества)
testX (содержит признаки для тестового множества)

Первый столбец содержит порядковый номер объекта, поэтому его рекомендуется сразу удалить

In [2]:
# ваш код
import pandas as pd
train_data = pd.read_csv('train.csv')
train_data = train_data.drop(columns=['Id'])
trainX = train_data.drop(columns=['SalePrice'])
trainY = train_data['SalePrice']
testX = pd.read_csv('test.csv')
testX = testX.drop(columns=['Id'])
print(trainX.shape, trainY.shape, testX.shape)

(1460, 79) (1460,) (1459, 79)


In [4]:
print(trainX.iloc[0])

MSSubClass           60
MSZoning             RL
LotFrontage        65.0
LotArea            8450
Street             Pave
                  ...  
MiscVal               0
MoSold                2
YrSold             2008
SaleType             WD
SaleCondition    Normal
Name: 0, Length: 79, dtype: object


### 2. Предварительная обработка данных

Как правило "реальные" данные содержат пропущенные значения и прочие нечисловые признаки, поэтому прежде чем запустить методы fit и  predict модели, необходимо сделать все признаки числовыми.

#### 2.1 Обработка пропущенных значений (2 балла)

Изучите раздел документации https://scikit-learn.org/stable/modules/impute.html#impute и параметры классов SimpleImputer и MissingIndicator

In [5]:
from sklearn.impute import MissingIndicator, SimpleImputer
import numpy as np

In [6]:
indicator = MissingIndicator()
train_mask_missing_values_only = indicator.fit_transform(trainX)
test_mask_missing_values_only = indicator.fit_transform(testX)
print(np.sum(train_mask_missing_values_only))
print(np.sum(test_mask_missing_values_only))

7829
7878


В данных были обнаружены пропущенные значение. 
При помощи класса MissingIndicator устраните пропущенные значения, использовав самый часто встречаемый элемент.

In [10]:
# ваш код

imp = SimpleImputer(strategy='most_frequent')
trainX_without_missing = imp.fit_transform(trainX)# ваш код
testX_without_missing = imp.fit_transform(testX)# ваш код

In [11]:
indicator = MissingIndicator()
train_mask_missing_values_only = indicator.fit_transform(trainX_without_missing)
test_mask_missing_values_only = indicator.fit_transform(testX_without_missing)
print(np.sum(train_mask_missing_values_only))
print(np.sum(test_mask_missing_values_only))

0
0


Пропущенные значения удалены.

In [13]:
print(trainX_without_missing)

[[60 'RL' 65.0 ... 2008 'WD' 'Normal']
 [20 'RL' 80.0 ... 2007 'WD' 'Normal']
 [60 'RL' 68.0 ... 2008 'WD' 'Normal']
 ...
 [70 'RL' 66.0 ... 2010 'WD' 'Normal']
 [20 'RL' 68.0 ... 2010 'WD' 'Normal']
 [20 'RL' 75.0 ... 2008 'WD' 'Normal']]


#### 2.2  Обработка категориальных значений (2 балла)

Изучите раздел документации https://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features

При помощи класса OrdinalEncoder удалите категориальные признаки.

In [14]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder

encoder = OrdinalEncoder()
encoder.fit(trainX_without_missing)
trainX_without_missing_and_cat = encoder.transform(trainX_without_missing)
encoder.fit(testX_without_missing)
testX_without_missing_and_cat = encoder.transform(testX_without_missing)


In [15]:
print(trainX_without_missing_and_cat[0])

[  5.   3.  36. 327.   1.   0.   3.   3.   0.   4.   0.   5.   2.   2.
   0.   5.   6.   4. 104.  53.   1.   1.  12.  13.   1. 128.   2.   4.
   2.   2.   3.   3.   2. 361.   5.   0.  73. 179.   1.   0.   1.   4.
 134. 232.   0. 531.   1.   0.   2.   1.   3.   1.   2.   6.   6.   0.
   2.   1.  89.   1.   2. 220.   4.   4.   2.   0.  49.   0.   0.   0.
   0.   2.   2.   2.   0.   1.   2.   8.   4.]


Теперь данные готовы для обучения моделей

Опишите достоинства и недостатки данного метода обработки категориальных признаков

ваш ответ


### 3. Разделение данных на обучение и валидацию (1 балл)

Разделите обучающее множество на обучение(75%) и валидацию(25%), воспользовавшись функцией train_test_split с random_state=42

In [16]:
# ваш код
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(trainX_without_missing_and_cat, trainY, test_size=0.25, random_state=42)

In [17]:
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(1095, 79) (365, 79) (1095,) (365,)


### 4. Обучение моделей (4 балла)

Данные готовы для обучения алгоритмов машинного обучения. В качестве базовой можеди возьмём линейную регрессию, меткрика качества - mean_squared_log_error

In [23]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_log_error

In [24]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_val)
y_pred[y_pred < 0.0] = 0.0

print(mean_squared_log_error(y_val, y_pred))

0.3823690756085036


Текущую модель можно существенно улучшить. Добейтесь на валидации ошибки меньше 0.03 без каких-либо ограничений на алгоритмы

In [25]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_val)
print(mean_squared_log_error(y_val, y_pred))


0.020837777498121827


### 5. Способы улучшения качества модели

1. Применить другую стратегию обработки пропущенных значений из scikit-learn (1 балл + 1 балл, если получили прирост качества для алгоритма с ошибкой меньше 0.03 )

2. Добавить бинарные признаки отсутствия/присутствия значения для столбцов, где есть пропущенные значения (1 балл)

3. Реализовать самостоятельно одну из стратегий обработки пропущенных значений из семинара (начиная с kNN) (2 балла)

4. Применить стратегию OneHotEncoder  для обработки категориальных признаков (2 балла)

5. Применить стратегию агрегированния данных для обработки категориальных признаков (2 балла)

Реализуйте не менее двух идей из предложенных выше

In [26]:
# 1 пункт:(для knn)
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error

n_neighbors = 5
knn_imputer = KNNImputer(n_neighbors=n_neighbors)

encoder = OrdinalEncoder()
encoder.fit(trainX)
X_train_without_cat = encoder.transform(trainX)

X_train_imputed_without_missing_and_cat = knn_imputer.fit_transform(X_train_without_cat)
X_train, X_val, y_train, y_val = train_test_split(X_train_imputed_without_missing_and_cat, trainY, test_size=0.25, random_state=42)


rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_val)
print(mean_squared_log_error(y_val, y_pred))
#Было 0.020837777498121827 -> получили прирост качества на 0.0006 для алгоритма с ошибкой меньше 0.03 

0.020280369927036395


In [28]:
# 2 пункт:
import pandas as pd
X_train_copy = trainX.copy()
for column in trainX.columns:
    if X_train_copy[column].isnull().any():
        X_train_copy[f"{column}_flmissval"] = X_train_copy[column].isnull().astype(int)
X_train_copy

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,Electrical_flmissval,FireplaceQu_flmissval,GarageType_flmissval,GarageYrBlt_flmissval,GarageFinish_flmissval,GarageQual_flmissval,GarageCond_flmissval,PoolQC_flmissval,Fence_flmissval,MiscFeature_flmissval
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,1,0,0,0,0,0,1,1,1
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,0,0,1,1,1
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,0,0,1,1,1
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,0,0,0,0,0,1,1,1
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,0,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,0,0,1,1,1
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,0,0,1,0,1
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,0,0,1,0,0
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,1,0,0,0,0,0,1,1,1
